# Scop3P

A comprehensive database of human phosphosites within their full context. Scop3P integrates sequences (UniProtKB/Swiss-Prot), structures (PDB), and uniformly reprocessed phosphoproteomics data (PRIDE) to annotate all known human phosphosites. 

Scop3P, available at https://iomics.ugent.be/scop3p, presents a unique resource for visualization and analysis of phosphosites and for understanding of phosphosite structure–function relationships.

## Dependencies

In [ ]:
%%capture
!pip install pandas matplotlib py3Dmol b2btools==3.0.7b2

In [ ]:
%%capture
import tempfile
import requests
import pandas as pd 
import py3Dmol
from b2bTools import SingleSeq, constants

## Target protein

In [ ]:
TARGET_PROTEIN_ID = "O00571" # Write here the Protein ID of your protein of interest
PDB_ID = "4PXA" # Write here the PDB ID of your protein of interest

## API Request

This function makes a GET request to SCOP3P API endpoint for a given protein accession ID and returns the protein sequence in string format.

In [ ]:
def fetch_sequence_aminoacids(accession):
    BASE_URL = f"http://uniprot.org/uniprotkb/{accession}.fasta"
    url = f'{BASE_URL}?accession={accession}'
    response = requests.get(url)
    if response.status_code == 200:
        raw_fasta_sequence = response.content.decode("utf-8")
    else:
        raw_fasta_sequence = ""
    
    lines = raw_fasta_sequence.split('\n')
    protein_id = str(lines[0])
    amino_acids = "".join([str(l) for l in lines[1:]])
    
    return protein_id, amino_acids

def fetch_protein_modifications(accession):
    """
    Fetches protein modifications for a given UniProt ID.

    Parameters:
    accession (str): UniProt ID of the protein.

    Returns:
    dict: A dictionary containing protein modifications.
    """
    BASE_URL = "https://iomics.ugent.be/scop3p/api/modifications"
    url = f'{BASE_URL}?accession={accession}'
    headers = {'accept': 'application/json'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return None


## Data parsing

For parsing the JSON as a table, we'll use pandas library.

In [ ]:
scop3P_results = fetch_protein_modifications(TARGET_PROTEIN_ID)
_protein_id, sequence = fetch_sequence_aminoacids(TARGET_PROTEIN_ID)

protein_name = scop3P_results['proteinName']
entry_name = scop3P_results['entryName']
accession = scop3P_results['accession']
url = scop3P_results['url']
modifications = scop3P_results['modifications']

print("""
--------------------------------------------------------------------------------
Scop3P: A Comprehensive Resource of Human Phosphosites within Their Full Context
--------------------------------------------------------------------------------

{0}:{1}

Phospho-sites found: {3} entries.

Full entry available on SCOP3P website: {2}
""".format(entry_name, protein_name, url, len(modifications)))

In [ ]:
def get_modifications_table(modifications):
    """
    Displays the protein modifications in a pandas DataFrame.

    Parameters:
    modifications (list): A list of dictionaries, each representing a protein modification.
    """
    df = pd.DataFrame(modifications)
    df = df[['residue', 'name', 'evidence', 'position', 'source', 'reference', 'functionalScore', 'specificSinglyPhosphorylated']]
    
    return df 
    
modifications_table = get_modifications_table(modifications)
display(modifications_table)

## Predict biophysical features

In [ ]:
%%capture
def predict_biophysical_features(accession, sequence):
    with tempfile.NamedTemporaryFile(prefix="seq_", suffix=".fasta", mode="w") as fp:
        fp.write(f">{accession}\n{sequence}\n")
        fp.flush()
        fp.seek(0)
        
        pred = SingleSeq(fp.name).predict(tools=[constants.TOOL_DYNAMINE, constants.TOOL_DISOMINE, constants.TOOL_EFOLDMINE]).get_all_predictions()
    
    return pred

biophysical_features = predict_biophysical_features(TARGET_PROTEIN_ID, sequence)
biophysical_features_target_protein = biophysical_features['proteins'][TARGET_PROTEIN_ID]

In [ ]:
# values above 1.0 membrane spanning regions, 
# Values above 0.8 indicate rigid conformations, 
# Values between 0.69-0.80 are 'context' dependent and capable of being either rigid or flexible.
# values below 0.69 flexible regions. 

phosphorylated = list(modifications_table['position'])
biophysical_features_target_protein['backbone_interpretation'] = []
biophysical_features_target_protein['disorder_propensity_interpretation'] = []
biophysical_features_target_protein['early_folding_interpretation'] = []
biophysical_features_target_protein['phosphorilation'] = []

for index, residue in enumerate(biophysical_features_target_protein['seq']):
    current_backbone = biophysical_features_target_protein['backbone'][index]
    biophysical_features_target_protein['backbone_interpretation'].append(1 if current_backbone > 1 else 2 if current_backbone > 0.8 else 3 if current_backbone > 0.69 else 4)

    current_disorder_propensity = biophysical_features_target_protein['disoMine'][index]
    biophysical_features_target_protein['disorder_propensity_interpretation'].append(1 if current_disorder_propensity > 0.5 else 0)

    current_early_folding = biophysical_features_target_protein['earlyFolding'][index]
    biophysical_features_target_protein['early_folding_interpretation'].append(1 if current_disorder_propensity > 0.169 else 0)
    
    biophysical_features_target_protein['phosphorilation'].append(1 if index + 1 in phosphorylated else 0)
    

## Plotting results

For visualizing the 3D structure, we'll use py3Dmol library.

In [ ]:
import matplotlib.pyplot as plt

# plot:
fig, ax = plt.subplots()
fig.set_figwidth(15)

x_len = len(biophysical_features_target_protein['seq'])

# Phospho

xs = [0.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['phosphorilation'][i] == 0]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#bab9b7', linelengths=1) # unphosphorylated

xs = [0.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['phosphorilation'][i] == 1]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#ba4d4a', linelengths=1) # phosphorylated

# Disorder

xs = [1.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['disorder_propensity_interpretation'][i] == 0]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#8874a3', linelengths=1) # Ordered
 
xs = [1.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['disorder_propensity_interpretation'][i] == 1]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#a34054', linelengths=1) # Disorder

# Backbone dynamics 

xs = [2.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['backbone_interpretation'][i] == 1]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#cac919', linelengths=1) # membrane spanning 

xs = [2.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['backbone_interpretation'][i] == 2]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#ffa700', linelengths=1) # rigid

xs = [2.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['backbone_interpretation'][i] == 3]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#cac919', linelengths=1) # context dependent

xs = [2.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['backbone_interpretation'][i] == 4]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#27bd2f', linelengths=1) # flexible


# Early folding

xs = [3.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['early_folding_interpretation'][i] == 0]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#5779b2', linelengths=1) # early folding

xs = [3.5]
vs = [[i for i, r in enumerate(biophysical_features_target_protein['seq']) if biophysical_features_target_protein['early_folding_interpretation'][i] == 1]]
ax.eventplot(vs, orientation="horizontal", lineoffsets=xs, color='#54b2a9', linelengths=1) # non-early folding

ax.set(
    xlim=(0, x_len),
    ylim=(0, 4), yticks=[0.5, 1.5, 2.5, 3.5]
)

ax.set_xlabel('Positions')
ax.set_yticklabels(['amino acids', 'disordered propensity', 'backbone dynamics', 'early folding'])
ax.minorticks_off()
ax.autoscale()
plt.show()